In [1]:
import numpy as np

In [2]:
def print_grid(grid, min_x):
    for y in range(grid.shape[1]):
        for x in range(min_x, grid.shape[0]):
            if grid[x, y] == 0:
                print('.', end='')
            elif grid[x, y] == 2:
                print('~', end='')
            elif grid[x, y] == 3:
                print('|', end='')
            elif grid[x, y] == 1:
                print('#', end='')
            else:
                print(grid[x, y], end='')
        print()

In [3]:
xvals = []
yvals = []

with open('input.txt') as f:
    for line in f:
        line = line.strip()
        if not line:
            break
        left, right = line.split(' ')
        left_val = int(left[2:-1])
        
        right_first_val, right_last_val = list(map(int, right[2:].split('..')))
        if left[0] == 'x':
            xvals.append(int(left[2:-1]))
            yvals.extend(range(right_first_val, right_last_val + 1))
        else:
            yvals.append(int(left[2:-1]))
            xvals.extend(range(right_first_val, right_last_val + 1))

grid = np.zeros((max(xvals) + 10, max(yvals) + 1), dtype=int)

with open('input.txt') as f:
    for line in f:
        line = line.strip()
        if not line:
            break
        left, right = line.split(' ')
        left_val = int(left[2:-1])
        
        right_first_val, right_last_val = list(map(int, right[2:].split('..')))
        if left[0] == 'x':
            grid[left_val, right_first_val:right_last_val+1] = 1
        else:
            grid[right_first_val:right_last_val+1, left_val] = 1


def do_downpour(grid, spout):
    cur_pour_x, cur_pour_y = spout
    
    if cur_pour_y + 1 >= grid.shape[1]:
        # We've hit the bottom of the screen!
        return True

    hit_bottom = False
    while grid[cur_pour_x, cur_pour_y + 1] == 0:
        grid[cur_pour_x, cur_pour_y + 1] = 3
        cur_pour_y += 1
        if cur_pour_y + 1>= grid.shape[1]:
            # We've hit the bottom of the screen!
            hit_bottom = True
            break
    if hit_bottom:
        return True
        
    # Now we've hit clay or water. If the water downpours, we can stop
    if grid[cur_pour_x, cur_pour_y + 1] == 3:
        return True
    
    # Now we've either hit clay or water that doesn't downpour, which is
    # the same. Keep up filling until we spill over
    while True:
        left_x = cur_pour_x
        found_downpour_left = found_downpour_right = False

        while grid[left_x - 1, cur_pour_y] != 1:
            left_x -= 1

            assert grid[left_x, cur_pour_y] != 3
            
            # If I hit water, then just go ahead and skip.
            # since this wasn't a downpour
            if grid[left_x, cur_pour_y] == 2:
                continue
            
            # We have empty space left. It gets filled and we
            # check if it has a potential downpour
            grid[left_x, cur_pour_y] = 2
            if grid[left_x, cur_pour_y + 1] == 0:
                # We've found our new downpour on the left.
                # Attempt to go down it
                found_downpour_left = do_downpour(grid, (left_x, cur_pour_y))
                if found_downpour_left:
                    break
            elif grid[left_x, cur_pour_y + 1] == 3:
                # We found a downpour already! We're done!
                found_downpour_left = True
                break
                
        # We *always* need to check to the right at least for this round
        right_x = cur_pour_x
        while grid[right_x + 1, cur_pour_y] != 1:
            right_x += 1
            
            # We can't hit a downspout or else we'd be flooded already
            assert grid[right_x, cur_pour_y] != 3
            
            # We can't hit plain water because we're DFSing to the left
            assert grid[right_x, cur_pour_y] != 2
            
            # So we're left with empty space. It gets filled and we check for potential downpours
            grid[right_x, cur_pour_y] = 2
            if grid[right_x, cur_pour_y + 1] == 0:
                # We've found our new downpour on the right
                found_downpour_right = do_downpour(grid, (right_x, cur_pour_y))
                if found_downpour_right:
                    break
            elif grid[right_x, cur_pour_y + 1] == 3:
                found_downpour_right = True
                break

        found_downpour = found_downpour_left | found_downpour_right
        if found_downpour:
            grid[left_x:right_x + 1, cur_pour_y] = 3
            return True
        else:
            grid[left_x:right_x + 1, cur_pour_y] = 2

        # The water rises
        cur_pour_y -= 1
        if cur_pour_y == spout[1]:
            # The water rose to the original level. This is no bueno
            return False
        
do_downpour(grid, (500, 0))

True

In [4]:
print('Answer to part 1:', (grid[:, min(yvals):] > 1).sum())
print('Answer to part 2:', (grid[:, min(yvals):] == 2).sum())

Answer to part 1: 31412
Answer to part 2: 25857


In [5]:
from matplotlib import pyplot as plt
plt.imsave('coolgraph.tif', grid.T)